In [ ]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 35kB/s 
     |████████████████████████████████| 460kB 39.8MB/s 
     |████████████████████████████████| 3.0MB 48.7MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
# Check Tensorflow version again
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
from google.colab import files
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
uploaded = files.upload()

Saving davydata1005.json to davydata1005.json


In [ ]:
!ls *.json

davydata1005.json


In [ ]:
file_name = "davydata1005.json"
data = uploaded[file_name]

In [ ]:
mylist = ["suivi_de_ma_candidature", "cdi", "offres_dav", "depot_candidature", "recrutement_stagiaire",
          "recrutement_varié", "postes_managers", "processus_recrutement",
          "appskills", "besoin_client", "demande_formation", "journee_chez_dav", "renvoi_vers_google", "oui_next", "autre_question", "bien_next",
          "soit_next", "comment_puis_je_aider", "stop", "merci", "aurevoir", "signalement_erreur", "parlez_de_vous", "avis_dav",
          "bcorp_gptw", "clients_dav", "colorz", "dav_en_chiffres", "differentiateurs_dav", "dvc", "expertises_de_dav", "filiales_de_dav",
          "futurs_developpements_int", "historique", "logo", "news_dav", "on_ne_parle_pas_valeurs", 
          "projets_de_davidson", "r&d", "secret_mais_contactez_nous",
          "secret_pour_toujours", "sysk", "turnover", "twister", "management_evolutions",
          "contact_board", "contact_ceo", "smartindus", "data_management", "healthcare",
          "network_digital", "finance", "presentation_davy", "chiffre_42", "musique", "ref_biblio", "gaming", "insulte_davy",
          "blague","horoscope", "les_partenariats_dav", "cherche_dans_l_orga",
          "meteo","rappelez_moi","demande_partenariat", "coworking", "concurrent_dav", "beurk", "dutler_video", "dutler_group",
          "campus", "pourquoi_un_bot", "non", "brian_is_in_the_kitchen", "discord", "sorry", "offres_vie", "Pas_possible"]

In [ ]:
d = json.loads(data)
d[1932]

{'classe': 'management_evolutions',
 'question': 'comment accompagnez vous vos consultants'}

In [ ]:
len(mylist)

78

In [ ]:
i = 0
map_dict = {}
for elm in mylist:
  map_dict.update({elm: i})
  i+=1

map_dict

{'Pas_possible': 77,
 'appskills': 8,
 'aurevoir': 20,
 'autre_question': 14,
 'avis_dav': 23,
 'bcorp_gptw': 24,
 'besoin_client': 9,
 'beurk': 67,
 'bien_next': 15,
 'blague': 58,
 'brian_is_in_the_kitchen': 73,
 'campus': 70,
 'cdi': 1,
 'cherche_dans_l_orga': 61,
 'chiffre_42': 53,
 'clients_dav': 25,
 'colorz': 26,
 'comment_puis_je_aider': 17,
 'concurrent_dav': 66,
 'contact_board': 45,
 'contact_ceo': 46,
 'coworking': 65,
 'data_management': 48,
 'dav_en_chiffres': 27,
 'demande_formation': 10,
 'demande_partenariat': 64,
 'depot_candidature': 3,
 'differentiateurs_dav': 28,
 'discord': 74,
 'dutler_group': 69,
 'dutler_video': 68,
 'dvc': 29,
 'expertises_de_dav': 30,
 'filiales_de_dav': 31,
 'finance': 51,
 'futurs_developpements_int': 32,
 'gaming': 56,
 'healthcare': 49,
 'historique': 33,
 'horoscope': 59,
 'insulte_davy': 57,
 'journee_chez_dav': 11,
 'les_partenariats_dav': 60,
 'logo': 34,
 'management_evolutions': 44,
 'merci': 19,
 'meteo': 62,
 'musique': 54,
 'netw

In [ ]:
len(map_dict)

78

In [ ]:
vocab_size = 15000
embedding_dim = 16
max_length = 12
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 0.999
import pickle
sentences = []
labels = []
data = []
data = d

def preprocess_select_data(data):

    for elm in data:
        sentences.append(elm["question"])
        labels.append(elm["classe"])
    return sentences, labels

def solution_model():
    sentences, labels = preprocess_select_data(data)
    sentences = sentences +  sentences
    labels = labels + labels

    #shuffle
    import random
    c = list(zip(sentences, labels))
    random.shuffle(c)
    sentences, labels = zip(*c)
    categorized_labels = []
    for l in labels:
      categorized_labels.append(map_dict[l])
    categorized_labels = tf.keras.utils.to_categorical(categorized_labels, num_classes=78, dtype='float32')


    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,)

    
    tokenizer.fit_on_texts(sentences)
    word_index = tokenizer.word_index
    training_sentences = sentences[0:int(training_size*len(sentences))]
    testing_sentences = sentences[int(training_size*len(sentences)):]
    training_labels = categorized_labels[0:int(training_size*len(sentences))]
    testing_labels = categorized_labels[int(training_size*len(sentences)):]

    training_sequences = tokenizer.texts_to_sequences(training_sentences)

    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

    
    # saving
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    training_padded = np.array(training_padded) 
    training_labels = np.array(training_labels)
    testing_padded = np.array(testing_padded)
    testing_labels = np.array(testing_labels)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(64, 5, activation="relu"),
        tf.keras.layers.MaxPool1D(4),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(78, activation='softmax')
    ])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(training_padded, training_labels, epochs=100, validation_data=(testing_padded, testing_labels))

    # Plot loss per iteration
    
    # import matplotlib.pyplot as plt
    """
    plt.plot(history.history['loss'], label='training loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    
    # Plot accuracy per iteration
    plt.plot(history.history['accuracy'], label='training acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.legend()
    """
    return model, word_index
    
    
if __name__ == '__main__':
    model, word_index = solution_model()
    model.save("Davy.h5")


Epoch 1/100
729/729 [==============================] - 7s 9ms/step - loss: 3.5114 - accuracy: 0.1439 - val_loss: 3.6871 - val_accuracy: 0.0833
Epoch 2/100
729/729 [==============================] - 6s 8ms/step - loss: 2.2718 - accuracy: 0.4063 - val_loss: 2.2274 - val_accuracy: 0.2917
Epoch 3/100
729/729 [==============================] - 6s 8ms/step - loss: 1.5799 - accuracy: 0.5922 - val_loss: 1.5072 - val_accuracy: 0.5000
Epoch 4/100
729/729 [==============================] - 6s 8ms/step - loss: 1.2073 - accuracy: 0.6931 - val_loss: 1.1256 - val_accuracy: 0.6667
Epoch 5/100
729/729 [==============================] - 6s 8ms/step - loss: 0.9771 - accuracy: 0.7547 - val_loss: 0.9837 - val_accuracy: 0.7083
Epoch 6/100
729/729 [==============================] - 6s 8ms/step - loss: 0.8288 - accuracy: 0.7928 - val_loss: 0.7502 - val_accuracy: 0.9167
Epoch 7/100
729/729 [==============================] - 6s 8ms/step - loss: 0.7230 - accuracy: 0.8209 - val_loss: 0.8311 - val_accuracy: 0.7917

In [ ]:
len(word_index)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 12, 16)            240000    
_________________________________________________________________
dropout (Dropout)            (None, 12, 16)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 8, 64)             5184      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 78)                5070      
Total params: 283,278
Trainable params: 283,278
Non-trainable params: 0
__________________________________________________

In [ ]:
import string
question = "Quelles sont les partenaires Davidson ?"
print(len(word_index))

def remove_punctuation(question):
    """ TO DO
    """  
    string.punctuation
    question = question.replace('\\', '')
    question = question.replace('-', ' ')
    question_fi = question.lower().strip(string.punctuation)
    processed_question = ' '.join(word.strip(string.punctuation) for word in question_fi.split())
    return processed_question


5293


In [ ]:
## Prediction updated
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


new_model = tf.keras.models.load_model('Davy.h5')
question = "Hello"
processed_question = remove_punctuation(question)
print(processed_question)
training_sequences = tokenizer.texts_to_sequences([processed_question])
padded = pad_sequences(training_sequences, padding=padding_type, maxlen=max_length)
classes = new_model.predict(padded)
predicted_indice = np.argmax(classes[0])
confidence_score = classes[0][predicted_indice]

inv_map = {v: k for k, v in map_dict.items()}
predicted_class = inv_map[predicted_indice]
print(predicted_class)
print(confidence_score)

FileNotFoundError: ignored

In [ ]:
len(inv_map)
inv_map

{0: 'suivi_de_ma_candidature',
 1: 'cdi',
 2: 'offres_dav',
 3: 'depot_candidature',
 4: 'recrutement_stagiaire',
 5: 'recrutement_varié',
 6: 'postes_managers',
 7: 'processus_recrutement',
 8: 'appskills',
 9: 'besoin_client',
 10: 'demande_formation',
 11: 'journee_chez_dav',
 12: 'renvoi_vers_google',
 13: 'oui_next',
 14: 'autre_question',
 15: 'bien_next',
 16: 'soit_next',
 17: 'comment_puis_je_aider',
 18: 'stop',
 19: 'merci',
 20: 'aurevoir',
 21: 'signalement_erreur',
 22: 'parlez_de_vous',
 23: 'avis_dav',
 24: 'bcorp_gptw',
 25: 'clients_dav',
 26: 'colorz',
 27: 'dav_en_chiffres',
 28: 'differentiateurs_dav',
 29: 'dvc',
 30: 'expertises_de_dav',
 31: 'filiales_de_dav',
 32: 'futurs_developpements_int',
 33: 'historique',
 34: 'logo',
 35: 'news_dav',
 36: 'on_ne_parle_pas_valeurs',
 37: 'projets_de_davidson',
 38: 'r&d',
 39: 'secret_mais_contactez_nous',
 40: 'secret_pour_toujours',
 41: 'sysk',
 42: 'turnover',
 43: 'twister',
 44: 'management_evolutions',
 45: 'contac

In [ ]:
word_index
with open('example.txt', 'w') as f:
  f.write(str(word_index))

files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(word_index)

5645